# Markov Decision Process (MDP)
---

MDP is a mathematical method to define Reinforcement Learning problems. It is generally composed of a tuple, $<\cal{S}, \cal{A}, P, R, \gamma>$. <br>
Let us see each elements of MDP. <br>
Example: `GridworldEnv` (made by inheriting `discrete.DiscreteEnv` of OpenAI `gym`) <br>
OpenAI `gym` is used as the base class for several Reinforcement Learning environments since it provides a standardized interface which is appropriate to be used for Reinforcement Learning.

In [1]:
import sys; sys.path.append('..') # add project root to the python path

In [2]:
# Gridworld Environment
from src.common.gridworld import GridworldEnv

`4 X 4 Gridworld` 

The visualized MDP environment is as follows.

```
===========
T  x  o  o
o  o  o  o
o  o  o  o
o  o  o  T
===========
```

T: destination (Terminal state) <br>
x: current location $s_t$<br>
o: points in other environments

In [3]:
num_y, num_x = 4, 4
# instantiate the environment above
env = GridworldEnv(shape =  [num_y, num_x])

## State Space $\cal{S}$ and Action (or State-action) Space $\cal{A}$

The State Space in the above is composed of 16 states divided by a 4 X 4 grid. <br>
The Action Space, $\cal{A}$, consists of the following 4 actions in all states $s$.

>`up`, `rihgt`, `down`, `left`  <br>

Once an agent moves outside the grid (`action`), its location does not change. <br>
For example, if it decides to move `upside` or `to the right` from the highest and the far right state in the grid, then its location does not change.

In [4]:
observation_space = env.observation_space
action_space = env.action_space
print(f"Number of states: {observation_space}")
print(f"Number of actions: {action_space}")

Number of states: Discrete(16)
Number of actions: Discrete(4)


## State Transition Matrix $P$ and Reward Function $R$

A state transition matrix, $P$, in MDP is not a matrix. Rather, it is a `tensor`.

### Tensor

<img src="./images/tensor.png" width="100%" height="50%" title="px(픽셀) 크기 설정" alt="Tensor"></img>

n-dimensional datatype: Rank `n` tensor or `n`-dimensional tensor <br>
(`n`: the required number of indices to have access to certain data in a tensor)

In [5]:
import numpy as np

In [6]:
# generating a rank-2 tensor
num_row = 2
num_col = 2
rank2_tensor = np.random.random(size = (num_row, num_col))

In [7]:
# able to infer the number of rank2_tensor's tensor
# by counting the number of square brackets
print(rank2_tensor)

[[0.45844718 0.50943114]
 [0.54694491 0.35885653]]


In [8]:
# numpy_array.shape() returns the number of indices for each dimension.
tensor_shape = rank2_tensor.shape
tensor_rank = len(tensor_shape)
print(f"Tensor shape: {tensor_shape}")
print(f"Tensor rank: {tensor_rank}")

Tensor shape: (2, 2)
Tensor rank: 2


## State Transition Matrix (Tensor) $P$ 

A state transition matrix, $P$, is a rank-3 tensor. The first axis, the second axis, and the third axis indicate actions ($a$), ($s$), and ($s'$) respectively.

Indices 0, 1, 2, 3 are given to action ($a$) and they are movements corresponding to [up, right, down, left] respectively. <br>

The number of states ($s$) is 4 x 4 = 16. As such, indices are assigned to have 0, 1, .. , 15. 0 and 1 are the top left and the location to the right by 1 from the location 0. <br>

In [9]:
# loading the state transition tensor P
P = env.P_tensor
# (the number of actions, the number of states, the number of states)
print(f"P shape: {P.shape}")

P shape: (4, 16, 16)


In [10]:
# See if the tensor is generated as intended.
# Showing an array whose dimension is larger than 2 or 3 is not desirable
# so before get started fix an axis.
action_up_prob = P[0, :, :]

In [11]:
print(action_up_prob)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


`GridworldEnv` is designed for certain actions to deterministically affect states. This is the special MDP case. . (rather stochastically.. )


>That is, all elements (except for a single element) in each row of the state transition matrix, $P$, are all 0.0.


In this case, if an action is `up`, then the agent always moves upside. However, in general, actions are not likely to deterministically affect states. Rather, they tend to stochastically have influence on states. For example, the agent might go to other directions even though an actions is `up`.


However, for easier understanding of optimal policy and optimal value use deterministic `GridworldEnv` for the time being.

## Properties of State Transition Matrix

Each row of a state transition matrix is the probability that an agent moves from a certain state, $s$, to the next state, $s'$,. Accordingly, every element is not less than 0 and the sum of each row equals 1.

In [12]:
# See if every element is not less than 0.
action_up_prob >= 0

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

Or you can make sure that every element is not less than 0 by using programming (defensive programming). <br>
>Defensive Programming: a form of defensive design intended to ensure the continuing function of a piece of software under unforeseen circumstances, often used where high availability, safety, or security is needed

In [13]:
is_greater_than_0 = action_up_prob >= 0
is_all_greater_than_0 = is_greater_than_0.sum() == is_greater_than_0.size
is_all_greater_than_0

True

In [14]:
# See if the sum of each row equals 1.
action_up_prob.sum(axis = 1)
# or -> action_up_prob.sum(axis = -1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Reward Function $R$ 

$R$ is a Rank2 tensor (matrix). The vertical and horizontal axes are states and actions respectively.

The `GridworldEnv` above receives a reward as much as `-1` no matter where (location) the agent is or no matter which action the agent choose before it arrives at the terminal state.

In [15]:
R = env.R_tensor
print(R)

[[ 0.  0.  0.  0.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [ 0.  0.  0.  0.]]


## Episode of MDP

Episodes of the MDP are $<(s_0, a_0, r_0, s_1), (s_1, a_1, r_1, s_2), ..., (s_{t}, a_{t}, r_{t}, s_{t+1}),..., (s_{T-1}, a_{T-1}, r_{T-1}, s_{T})>$. Each tuple indicates the state ($s_t$), the action ($a_t$), the reward ($r_t$), and the next state ($s_{t+1}$) at time $t$. $T$ is the terminal time. 

However, in general, add the information, `done`, which checks if a certain state is terminal state to each tuple of an episode for RL convenient implementation of Reinforcement Learning algorithms.

Let us simulate episodes in the `GridworldEnv`.

### Policy, $\pi$, is necessary to simulate MDP.

Let us use the policy which selects every action with probability of 0.25 for the simulation.

In [16]:
# Reset the Gridworld.
_ = env.reset()
print(f"Current position index: {env.s}")

Current position index: 13


In [17]:
# For more intuitive visualization,
# replace the indices of actions with directions.
action_mapper = {
    0: 'UP',
    1: 'RIGHT',
    2: 'DOWN',
    3: 'LEFT'
}

In [18]:
step_counter = 0
# Use 'while' since the number of episodes is unknown (random).
while True:
    print(f"At t = {step_counter}")
    env._render()
    
    # current state
    cur_state = env.s
    # random policy -> an integer in [0, 4)
    action = np.random.randint(low = 0, high = 4)
    # applying a randomly chosen action by using 'step' method
    next_state, reward, done, info = env.step(action)
    
    print(f"state: {cur_state}")
    print(f"action: {action_mapper[action]}")
    print(f"reward: {reward}")
    print(f"the next state: {next_state}\n")
    step_counter += 1
    if done:
        break

At t = 0
T  o  o  o
o  o  o  o
o  o  o  o
o  x  o  T
state: 13
action: UP
reward: -1.0
the next state: 9

At t = 1
T  o  o  o
o  o  o  o
o  x  o  o
o  o  o  T
state: 9
action: UP
reward: -1.0
the next state: 5

At t = 2
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
state: 5
action: RIGHT
reward: -1.0
the next state: 6

At t = 3
T  o  o  o
o  o  x  o
o  o  o  o
o  o  o  T
state: 6
action: DOWN
reward: -1.0
the next state: 10

At t = 4
T  o  o  o
o  o  o  o
o  o  x  o
o  o  o  T
state: 10
action: UP
reward: -1.0
the next state: 6

At t = 5
T  o  o  o
o  o  x  o
o  o  o  o
o  o  o  T
state: 6
action: LEFT
reward: -1.0
the next state: 5

At t = 6
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
state: 5
action: LEFT
reward: -1.0
the next state: 4

At t = 7
T  o  o  o
x  o  o  o
o  o  o  o
o  o  o  T
state: 4
action: RIGHT
reward: -1.0
the next state: 5

At t = 8
T  o  o  o
o  x  o  o
o  o  o  o
o  o  o  T
state: 5
action: LEFT
reward: -1.0
the next state: 4

At t = 9
T  o  o  o
x  o  o  o
o  o  o

## Simulating various episodes

Even though the state transition matrix ($P$) of the `GridworldEnv` is deterministic, the visited states and the length of each episode can be different (despite of the same starting point) each other due to the stochastic policy function ($\pi$). Let us simulate these cases too.

In [19]:
def run_episode(env, s0):
    # Reset the Gridworld.
    _ = env.reset()
    
    step_counter = 0
    while True:
        action = np.random.randint(low = 0, high = 4)
        next_state, reward, done, info = env.step(action)
        
        step_counter += 1
        if done:
            break
    return step_counter

In [20]:
n_episodes = 10
s0 = 6

for i in range(n_episodes):
    len_ep = run_episode(env, s0)
    print(f"Episode {i} | Length of episode: {len_ep}")

Episode 0 | Length of episode: 56
Episode 1 | Length of episode: 7
Episode 2 | Length of episode: 6
Episode 3 | Length of episode: 1
Episode 4 | Length of episode: 38
Episode 5 | Length of episode: 10
Episode 6 | Length of episode: 1
Episode 7 | Length of episode: 4
Episode 8 | Length of episode: 7
Episode 9 | Length of episode: 12
